# Rendre visible la pollution de l'eau potable

Données disponibles :
* [Eau distribuée par commune gouv.fr](https://www.data.gouv.fr/fr/datasets/resultats-du-controle-sanitaire-de-leau-distribuee-commune-par-commune/)
* [Eau du robinet](https://www.data.gouv.fr/fr/datasets/resultats-du-controle-sanitaire-de-leau-du-robinet/)

## Eau distribuée par commune

__Extrait de la doc :__

Le jeu de données se compose de trois types de fichiers. Le lien entre le fichier des prélèvements (PLV) et celui des résultats d’analyse (RESULT) se fait par l’intermédiaire du code du prélèvement (champ ‘referenceprel’). Le fichier UDI_COM détaille le lien entre UDI et communes et permet de pouvoir reconstituer un accès aux résultats par commune ; il est relié au fichier PLV par l’intermédiaire du champ ‘cdreseau’, correspondant au code SISE-Eaux de l’unité de distribution.

### Récupération des données

In [1]:
# Pour gérer le jeu de données volumineux
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.7 MB/s eta 0:00:00


#### Via téléchargement s3

In [2]:
import requests
import duckdb
import dask.dataframe as dd
import pandas as pd
import time

In [3]:
start_time = time.time()

# Définition du lien de téléchargement de la base DuckDB
url_s3 = 'https://pollution-eau-s3.s3.fr-par.scw.cloud/prod/database/data.duckdb'

# Element de sortie
local_file = 'data.duckdb'

# Récupérer la BD, puis l'écrire en local
response = requests.get(url_s3)
with open(local_file, 'wb') as file:
  file.write(response.content)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

Temps d'exécution : 52.61713266372681 seconds


In [4]:
# Connexion à la base de données
con = duckdb.connect(database='data.duckdb', read_only=True)

# Listing des tables chargées
tables_df = con.execute("SHOW TABLES").df()
print(tables_df)

# Fermeture de la connexion à la base de données
con.close()

               name
0      edc_communes
1  edc_prelevements
2     edc_resultats


##### Pandas dataframe

In [11]:
# Récupération des données de la table edc_communes
df_com = con.execute('SELECT * FROM edc_communes').df()
df_com.head()

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2020,2025-02-12
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2020,2025-02-12
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2020,2025-02-12
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2020,2025-02-12
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2020,2025-02-12


In [12]:
df_com.shape

(247622, 8)

In [13]:
# Récupération des données de la table edc_prelevements
df_plv = con.execute('SELECT * FROM edc_prelevements').df()
df_plv.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
1,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
2,001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
3,001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
4,001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12


In [14]:
df_plv.shape

(2083345, 20)

In [ ]:
# /!\ FAIT PLANTER LA SESSION, PLUS DE RAM DISPO /!\

# Récupération des données de la table edc_resultats
df_res = con.execute('SELECT * FROM edc_resultats').df()
df_res.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df_res.shape

In [ ]:
# Fermeture de la connexion à la base de données
con.close()

##### Dask dataframes

In [5]:
start_time = time.time()

# Connexion à la base de données
con = duckdb.connect(database='data.duckdb', read_only=True)

df_com = con.execute('SELECT * FROM edc_communes').fetchdf()

# Fermeture de la connexion à la base de données
con.close()

# Création du dataframe Dask
ddf_com = dd.from_pandas(df_com)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {ddf_com.shape[0].compute()}, colonnes : {ddf_com.shape[1]}')

# Premières lignes
ddf_com.head()

Temps d'exécution : 0.48 seconds
Lignes : 247622, colonnes : 8


,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2020,2025-02-12
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2020,2025-02-12
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2020,2025-02-12
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2020,2025-02-12
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2020,2025-02-12


In [6]:
start_time = time.time()

# Connexion à la base de données
con = duckdb.connect(database='data.duckdb', read_only=True)

df_plv = con.execute('SELECT * FROM edc_prelevements').fetchdf()

# Fermeture de la connexion à la base de données
con.close()

# Création du dataframe Dask
ddf_plv = dd.from_pandas(df_plv)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {ddf_plv.shape[0].compute()}, colonnes : {ddf_plv.shape[1]}')

# Premières lignes
ddf_plv.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Temps d'exécution : 75.25 seconds
Lignes : 2083345, colonnes : 20


,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
1,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
2,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
3,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
4,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12


In [ ]:
# /!\ FAIT PLANTER LA SESSION, PLUS DE RAM DISPO /!\

start_time = time.time()

# Connexion à la base de données
con = duckdb.connect(database='data.duckdb', read_only=True)

df_res = con.execute('SELECT * FROM edc_resultats').fetchdf()

# Fermeture de la connexion à la base de données
con.close()

# Création du dataframe Dask
ddf_res = dd.from_pandas(df_res)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {ddf_res.shape[0].compute()}, colonnes : {ddf_res.shape[1]}')

# Premières lignes
ddf_res.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

#### Via téléchargement en direct depuis gouv.fr, format .parquet

In [2]:
# Import des modules utilisés
import requests
import time
#import pandas as pd -> Dask utilisé car fichier RESULT très gros
import dask.dataframe as dd
#import zipfile
#import os

In [4]:
start_time = time.time()

# Définition des urls contenant les données, au format parquet
# URL COM_UDI
url_cu = 'https://www.data.gouv.fr/fr/datasets/r/211c23ea-60eb-4b3f-9000-ae319c16cfc9'
# URL RESULT
url_re = 'https://www.data.gouv.fr/fr/datasets/r/50f3cd73-4ce5-4c04-a45c-f883aae389be'
# URL PLV
url_pl = 'https://www.data.gouv.fr/fr/datasets/r/dd0f0c3a-49dc-4627-8d13-65b463041500'

# Définition des fichiers
output_cu = 'COM_UDI.parquet'
output_re = 'RESULT.parquet'
output_pl = 'PLV.parquet'

# Association url:fichier
dict_ = {url_cu:output_cu,
         url_re:output_re,
         url_pl:output_pl}

# Téléchargement
for i in range(len(dict_)):
  response = requests.get(list(dict_.keys())[i])
  with open(list(dict_.values())[i], 'wb') as f:
    f.write(response.content)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

Temps d'exécution : 24.61 seconds


In [5]:
start_time = time.time()

# Lecture du fichier COM_UDI
#df_cu = pd.read_parquet(output_cu, engine='pyarrow')
df_cu = dd.read_parquet(output_cu, ignore_metadata_file=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {df_cu.shape[0].compute()}, colonnes : {df_cu.shape[1]}')
# Affichage des premières lignes
df_cu.head()

Temps d'exécution : 0.08 seconds
Lignes : 444793, colonnes : 7


,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,annee
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2024
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2024
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2024


In [6]:
start_time = time.time()

# Lecture du fichier RESULT
#df_re = pd.read_parquet(output_re, engine='pyarrow')
df_re = dd.read_parquet(output_re, ignore_metadata_file=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {df_re.shape[0].compute()}, colonnes : {df_re.shape[1]}')
# Affichage des premières lignes
df_re.head()

Temps d'exécution : 0.04 seconds
Lignes : 112261092, colonnes : 18


,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,annee
0,001,00100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,<NA>,N,L,<10,µg/L,133,<NA>,<=200 µg/L,0.000000,7429-90-5,00100147233,2024
1,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147232,2024
2,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147233,2024
3,001,00100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,<NA>,N,T,"0,30",mg(Cl2)/L,165,<NA>,<NA>,0.300000,<NA>,<NA>,2024
4,001,00100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,<NA>,N,T,"0,34",mg(Cl2)/L,165,<NA>,<NA>,0.340000,<NA>,<NA>,2024


In [7]:
start_time = time.time()

# Lecture du fichier PLV
#df_pl = pd.read_parquet(output_pl, engine='pyarrow')
df_pl = dd.read_parquet(output_pl, ignore_metadata_file=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time:.2f} seconds")

# Taille du df
print(f'Lignes : {df_pl.shape[0].compute()}, colonnes : {df_pl.shape[1]}')
# Affichage des premières lignes
df_pl.head()

Temps d'exécution : 0.01 seconds
Lignes : 3696470, colonnes : 19


,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,annee
0,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
3,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
4,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024


#### Analyse df_cu
* Contient un mapping entre les communes et les cdr

In [8]:
df_cu.head()

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,annee
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2024
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2024
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2024


In [9]:
annees_unique = df_cu['annee'].unique()
print(annees_unique.compute())

0    2024
1    2023
2    2022
3    2021
4    2020
5    2019
6    2018
7    2017
8    2016
Name: annee, dtype: int32


#### Analyse df_pl
* Fichier recensant les prélèvements effectués

In [10]:
df_pl.head()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,annee
0,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
3,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
4,001,001000003,01007,AMBRONAY,<NA>,<NA>,<NA>,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024


In [11]:
annees_unique = df_pl['annee'].unique()
print(annees_unique.compute())

0    2023
1    2022
2    2018
0    2024
1    2021
2    2019
3    2016
0    2020
1    2017
Name: annee, dtype: int32


#### Analyse df_re

In [12]:
df_re.head()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,annee
0,001,00100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,<NA>,N,L,<10,µg/L,133,<NA>,<=200 µg/L,0.000000,7429-90-5,00100147233,2024
1,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147232,2024
2,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147233,2024
3,001,00100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,<NA>,N,T,"0,30",mg(Cl2)/L,165,<NA>,<NA>,0.300000,<NA>,<NA>,2024
4,001,00100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,<NA>,N,T,"0,34",mg(Cl2)/L,165,<NA>,<NA>,0.340000,<NA>,<NA>,2024


In [13]:
annees_unique = df_re['annee'].unique()
print(annees_unique.compute())

0    2023
0    2020
0    2018
1    2016
0    2024
1    2019
0    2021
0    2022
0    2017
Name: annee, dtype: int32


In [14]:
# Sélection sur l'année 2024 uniquement
condition_ = df_re['annee'] == 2024
df_re_2024 = df_re[condition_]
df_re_2024.head()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,annee
0,001,00100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,<NA>,N,L,<10,µg/L,133,<NA>,<=200 µg/L,0.000000,7429-90-5,00100147233,2024
1,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147232,2024
2,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),<NA>,O,L,Aspect normal,SANS OBJET,X,<NA>,<NA>,0.000000,<NA>,00100147233,2024
3,001,00100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,<NA>,N,T,"0,30",mg(Cl2)/L,165,<NA>,<NA>,0.300000,<NA>,<NA>,2024
4,001,00100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,<NA>,N,T,"0,34",mg(Cl2)/L,165,<NA>,<NA>,0.340000,<NA>,<NA>,2024


In [15]:
cdparametresiseeaux_uniques = df_re_2024['cdparametresiseeaux'].unique()
print(cdparametresiseeaux_uniques.compute())

0         CTF
1        SAVQ
2      BGPERY
3     BENFURA
4        BFNX
       ...   
86       MFTC
87    TEMPDOM
88    TETRACB
89    1112TCP
90       NONA
Name: cdparametresiseeaux, Length: 1338, dtype: string


[Paramètres à chercher](https://fr.wikipedia.org/wiki/Phtalate) (dans un premier temps) :
* di-2-éthylhexyle
  * DEHP
  * DOP
* di-isononyle : DINP
* di-isodécyle : DIDP
* benzylbutyle : BBP
* dibutyle : DBP
* diéthyle : DEP
* di-cyclohexyle
  * DCHP
  * DCP
* di-n-octyle : DNOP
* diméthyle : DMP

In [22]:
cdparametresiseeaux_uniques

Dask Series Structure:
npartitions=130
    string
       ...
     ...  
       ...
       ...
Dask Name: unique, 6 expressions
Expr=Unique(frame=(Filter(frame=ReadParquetFSSpec(86d9ada), predicate=ReadParquetFSSpec(86d9ada)['annee'] == 2024))['cdparametresiseeaux'])

In [25]:
to_check_court = ['DEHP', 'DOP', 'DINP', 'DIDP', 'BBP', 'DBP', 'DEP', 'DCHP',
                  'DCP', 'DNOP', 'DMP']

#are_present = cdparametresiseeaux_uniques.isin(to_check_court).compute()
#result_list = are_present.tolist()
are_present = [element in cdparametresiseeaux_uniques.compute().values for element in to_check_court]
print(f"Présence de {to_check_court} dans la liste ? {are_present}")

Présence de ['DEHP', 'DOP', 'DINP', 'DIDP', 'BBP', 'DBP', 'DEP', 'DCHP', 'DCP', 'DNOP', 'DMP'] dans la liste ? [True, False, False, False, False, True, False, False, True, False, False]


Phtalates retrouvés (sigle) : DEHP, DBP, DCP

In [26]:
to_check_long = ['di-2-éthylhexyle', 'di-2-ethylhexyle', 'di-isononyle',
                 'di-isodécyle', 'di-isodecyle', 'benzylbutyle', 'dibutyle',
                 'diéthyle', 'diethyle', 'di-cyclohexyle', 'di-n-octyle',
                 'diméthyle', 'dimethyle']

are_present = [element in cdparametresiseeaux_uniques.compute().values for element in to_check_long]
print(f"Présence de {to_check_long} dans la liste ? {are_present}")

Présence de ['di-2-éthylhexyle', 'di-2-ethylhexyle', 'di-isononyle', 'di-isodécyle', 'di-isodecyle', 'benzylbutyle', 'dibutyle', 'diéthyle', 'diethyle', 'di-cyclohexyle', 'di-n-octyle', 'diméthyle', 'dimethyle'] dans la liste ? [False, False, False, False, False, False, False, False, False, False, False, False, False]
